In [25]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import re
import lyricsgenius
import requests
#import api_key
from pathlib import Path
import pandas as pd
import time
from datetime import datetime
import xlsxwriter
import os
#import dataiku

In [20]:
client_access_token = "Mq5jqnkXjNX5A2fJF-j93_nVigeWhuSk8N4o2Necuz5ZskWOn7PSYTOhpest1Gc7"
LyricsGenius = lyricsgenius.Genius(client_access_token)
LyricsGenius.remove_section_headers = True

In [3]:
artist = "Taylor Swift"
artist = artist.replace(" ", "-")
artist

'Taylor-Swift'

# For one album

In [4]:
album_name_original = "Fearless (Taylor's Version)"

In [8]:
def clean_up(song_title):
    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")

    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")

    return clean_song_title

In [6]:
def title_contains_weird_strings(song_title) :
    pattern = r"Reputation Magazine|\[Poem\]|\[Prologue\]|\[Foreword\]|\[Liner Notes\]"
    return True if re.search(pattern, song_title) else False

In [9]:
def get_song_titles(album_name, artist) :
    # Clean album name and artist
    album_name = re.sub(r"\(Taylor's Version\)", "taylors-version", album_name)
    album_name = re.sub(r"[\[\]\(\)]", "", album_name)
    album_name = re.sub(r" ", "-", album_name)

    # Get the HTML page
    URL = f"https://genius.com/albums/{artist}/{album_name}"
    document = ''
    while document == '' :
        try :
            response = requests.get(URL)
            html_string = response.text
            document = BeautifulSoup(html_string, "html.parser")
            break
        except :
            print("Sleep now... Try connecting again")
            time.sleep(10)

    # Extract song titles and clean them
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    song_titles_legit = [song_title for song_title in song_titles if not title_contains_weird_strings(song_title)]
    return song_titles_legit

In [10]:
def get_clean_song_titles(song_titles) :
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)

    return clean_songs

In [11]:
song_titles = get_song_titles("reputation", artist)
song_titles

[]

In [26]:
URL = f"http://genius.com/albums/Taylor-swift/Reputation"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
response = requests.get(URL, headers=headers)
html_string = response.text
document = BeautifulSoup(html_string, "html.parser")
document


<!DOCTYPE html>

<html class="snarly song_stories_public_launch--enabled react_forums--disabled report_abuse--disabled" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<base href="//genius.com/" target="_top"/>
<script type="text/javascript">
//<![CDATA[

  var _sf_startpt=(new Date()).getTime();
  if (window.performance && performance.mark) {
    window.performance.mark('parse_start');
  }

//]]>
</script>
<title>Taylor Swift - reputation Lyrics and Tracklist | Genius</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="app-id=709482991" name="apple-itunes-app"/>
<link href="https://assets.genius.com/images/apple-touch-icon.png?1712086395" rel="apple-touch-icon"/>
<link href="https://assets.genius.com/images/apple-touch-icon.png?1712086395" rel="apple-touch-icon"/>
<!-- Mobile IE allows us to activate ClearType t

In [27]:
song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
song_title_tags

[<h3 class="chart_row-content-title">
               ...Ready for It?
               <span class="chart_row-content-title-subtitle">Lyrics</span>
 </h3>,
 <h3 class="chart_row-content-title">
               End Game (Ft. Ed Sheeran &amp; Future)
               <span class="chart_row-content-title-subtitle">Lyrics</span>
 </h3>,
 <h3 class="chart_row-content-title">
               I Did Something Bad
               <span class="chart_row-content-title-subtitle">Lyrics</span>
 </h3>,
 <h3 class="chart_row-content-title">
               Don't Blame Me
               <span class="chart_row-content-title-subtitle">Lyrics</span>
 </h3>,
 <h3 class="chart_row-content-title">
               Delicate
               <span class="chart_row-content-title-subtitle">Lyrics</span>
 </h3>,
 <h3 class="chart_row-content-title">
               Look What You Made Me Do
               <span class="chart_row-content-title-subtitle">Lyrics</span>
 </h3>,
 <h3 class="chart_row-content-title">
               

In [ ]:
clean_songs = get_clean_song_titles(song_titles)
clean_songs

In [ ]:
def create_track_numbers(clean_songs) :
    return [i for i in range(1, len(clean_songs)+1)]

In [ ]:
track_numbers = create_track_numbers(clean_songs)
track_numbers

In [ ]:
def get_featured_artists(song_title):
    if "Ft" in song_title :
        song_title = re.sub("\xa0", " ", song_title)
        matched = re.findall(r"\(Ft\.\s*(.*?)\)", song_title)
        featured_artist = matched[0]
    else :
        featured_artist = ''
    return featured_artist

In [ ]:
def get_featured_artists_in_one_album(album_name, artist) :
    song_titles = get_song_titles(album_name, artist)
    featured_artists = []
    for song_title in song_titles:
        featured_artist = get_featured_artists(song_title)
        featured_artists.append(featured_artist)

    return featured_artists

In [ ]:
get_featured_artists_in_one_album(album_name_original, artist)

In [ ]:
df = pd.DataFrame(columns=['Album', 'Song Title', 'Track Number', 'Featured Artist(s)'])
df['Song Title'] = clean_songs
df['Track Number'] = track_numbers
df['Featured Artist(s)'] = featured_artists
df['Album'] = album_name_original

df

## One song

In [ ]:
song1 = clean_songs[0]
song1

In [ ]:
song_dict =  LyricsGenius.search_song(title=song1, artist=artist, get_full_info=True).to_dict()
song_dict

In [ ]:
# Get necessary info
year = song_dict["release_date_components"]['year']
month = song_dict["release_date_components"]['month']
day = song_dict["release_date_components"]['day']
print(year, month, day)

In [ ]:
date_string = song_dict["release_date_for_display"]
parsed_date = datetime.strptime(date_string, "%B %d, %Y")
release_date = parsed_date.strftime("%Y-%m-%d")
print(release_date)

In [ ]:
url = song_dict["url"]
url

In [ ]:
lyric = song_dict["lyrics"]
lyric

## Accessing the complete site

In [ ]:
URL_one_song = url
response2 = requests.get(URL_one_song)
html_string2 = response2.text
document2 = BeautifulSoup(html_string2, "html.parser")
document2

In [ ]:
print(document2.prettify())

In [ ]:
song_info_tags = document2.find_all("div", attrs={"class": "SongInfo__Credit-nekw6x-3"})
print(len(song_info_tags))
song_info_tags

### Producers

In [ ]:
producers_tag = [tag for tag in song_info_tags if tag.find_all("div", string="Produced By")][0]
producers_tag

In [ ]:
producer_names_tag = producers_tag.find_all("a", attrs={"class": "StyledLink-sc-3ea0mt-0"})
producer_names_tag

In [ ]:
producer_names = [producer.text for producer in producer_names_tag]
producer_names

### Writers

In [ ]:
writers_tag = [tag for tag in song_info_tags if tag.find_all("div", string="Written By")][0]
writers_tag

In [ ]:
writer_names_tag = writers_tag.find_all("a", attrs={"class": "StyledLink-sc-3ea0mt-0"})
writer_names_tag

In [ ]:
writer_names = [writer.text for writer in writer_names_tag]
writer_names

### Backing vocals

In [ ]:
bg_vocals_tag = [tag for tag in song_info_tags if tag.find_all("div", string="Background Vocals")][0]
bg_vocals_tag

In [ ]:
bg_vocal_names_tag = bg_vocals_tag.find_all("a", attrs={"class": "StyledLink-sc-3ea0mt-0"})
bg_vocal_names_tag

In [ ]:
bg_vocal_names = [bg_vocal.text for bg_vocal in bg_vocal_names_tag]
bg_vocal_names

## Def

In [ ]:
def clean_lyrics(lyrics) :
    # Remove non-lyric part
    lyrics_cleaned = re.search(r'\sLyrics(.*?)\d*Embed', lyrics, re.DOTALL).group(1)

    # Remove the advertisement
    lyrics_clean = re.sub(r"See.*\$.*You might also like", '\n', lyrics_cleaned)

    return lyrics_clean

In [ ]:
def get_song_information(song_title, artist) :
    song_dict = ''
    while song_dict == '' :
        try :
            song_dict = LyricsGenius.search_song(title=song_title, artist=artist, get_full_info=True).to_dict()
            break
        except ConnectionError as e :
            print(e)
            print("Sleep now... Try connecting again")
            time.sleep(5)

    release_year = song_dict["release_date_components"]['year']
    release_month = song_dict["release_date_components"]['month']
    release_day = song_dict["release_date_components"]['day']

    date_string = song_dict["release_date_for_display"]
    parsed_date = datetime.strptime(date_string, "%B %d, %Y")
    release_date = parsed_date.strftime("%Y-%m-%d")

    url = song_dict["url"]
    lyrics = song_dict["lyrics"]
    return [release_date, release_year, release_month, release_day, url, clean_lyrics(lyrics)]

In [ ]:
def get_song_more_information(url_one_song) :

    document = ''
    while document == '' :
        try :
            response = requests.get(url_one_song)
            html_string = response.text
            document = BeautifulSoup(html_string, "html.parser")
            break
        except ConnectionError as e :
            print(e)
            print("Sleep now... Try connecting again")
            time.sleep(5)

    song_info_tags = document.find_all("div", attrs={"class": "SongInfo__Credit-nekw6x-3"})

    # Get producers
    producers_tag = [tag for tag in song_info_tags if tag.find_all("div", string="Produced By")][0]
    producer_names_tag = producers_tag.find_all("a", attrs={"class": "StyledLink-sc-3ea0mt-0"})
    producer_names = [producer.text for producer in producer_names_tag]

    # Get writers
    writers_tag = [tag for tag in song_info_tags if tag.find_all("div", string="Written By")][0]
    writer_names_tag = writers_tag.find_all("a", attrs={"class": "StyledLink-sc-3ea0mt-0"})
    writer_names = [writer.text for writer in writer_names_tag]

    # Get background vocals
    try :
        bg_vocals_tag = [tag for tag in song_info_tags if tag.find_all("div", string="Background Vocals")][0]
        bg_vocal_names_tag = bg_vocals_tag.find_all("a", attrs={"class": "StyledLink-sc-3ea0mt-0"})
        bg_vocal_names = [bg_vocal.text for bg_vocal in bg_vocal_names_tag]
    except :
        bg_vocal_names = ''
    return [producer_names, writer_names, bg_vocal_names]

In [ ]:
def get_additional_info(clean_songs) :
    df_info = pd.DataFrame(columns=['Release Date', 'Release Year', 'Release Month', 'Release Day', 'URL',
                                    'Lyrics', 'Producer(s)', 'Writer(s)', 'Background Vocal(s)'])
    for song in clean_songs :
        info1 = get_song_information(song, artist)
        url = info1[4]
        info2 = get_song_more_information(url)
        new_data = info1 + info2

        df_info.loc[len(df_info)] = new_data

    return df_info

In [ ]:
get_additional_info(clean_songs)

In [ ]:
df_complete = pd.concat([df, df_info], axis=1)
df_complete

# Loop all albums

In [ ]:
artist_id = "1177" # Artist ID of Taylor Swift

all_album_names = []
for page in range(1, 10) :
    album_list = LyricsGenius.artist_albums("1177", per_page=50, page=page)['albums']
    for album_dict in album_list :
        all_album_names.append(album_dict['name'])

all_album_names

In [ ]:
album_names_used = ["Taylor Swift",
                    "Fearless (Taylor's Version)",
                    "Speak Now (Taylor's Version)",
                    "Red (Taylor's Version)",
                    "1989 (Taylor's Version) [Deluxe]",
                    "reputation",
                    "Lover",
                    "folklore (deluxe version)",
                    "evermore (deluxe version)",
                    "Midnights (The Til Dawn Edition)",
                    "Midnights (The Late Night Edition)", # Get "You're Losing Me"
                    "The More Red (Taylor's Version) Chapter", # Get "Safe & Sound" and "Eyes Open"
                    "The More Fearless (Taylor's Version) Chapter", # Get "If This Was A Movie"
                    "The More Lover Chapter"#, # Get "All Of The Girls You Loved Before"
                    #"THE TORTURED POETS DEPARTMENT" # When the album is released!
                   ]

In [ ]:
def create_df_one_album(album, artist) :
    # Get all songs from one album
    song_titles = get_song_titles(album, artist)
    clean_songs = get_clean_song_titles(song_titles)
    featured_artists = get_featured_artists_in_one_album(album, artist)
    track_numbers = create_track_numbers(clean_songs)

    df = pd.DataFrame(columns=['Album', 'Song Title', 'Track Number', 'Featured Artist(s)'])
    df['Song Title'] = clean_songs
    df['Track Number'] = track_numbers
    df['Featured Artist(s)'] = featured_artists
    df['Album'] = album

    # Enrich the information of the songs

    df_info = get_additional_info(clean_songs)

    if len(df) == len(df_info) :
        df_complete = pd.concat([df, df_info], axis=1)
    else :
        print(f"The information for the album {album} can't be created. Different lengths of data.")

    return df_complete

In [ ]:
create_df_one_album("Taylor Swift", artist)

In [ ]:
create_df_one_album("Fearless (Taylor's Version)", artist)

In [ ]:
create_df_one_album("Speak Now (Taylor's Version)", artist)

In [ ]:
create_df_one_album("Red (Taylor's Version)", artist)

In [ ]:
create_df_one_album("1989 (Taylor's Version) [Deluxe]", artist)

In [ ]:
create_df_one_album("reputation", artist)

In [ ]:
create_df_one_album("Lover", artist)

In [ ]:
create_df_one_album("folklore (deluxe version)", artist)

In [ ]:
create_df_one_album("evermore (deluxe version)", artist)

In [ ]:
create_df_one_album("Midnights (The Til Dawn Edition)", artist)

In [ ]:
create_df_one_album("Midnights (The Late Night Edition)", artist)

In [ ]:
create_df_one_album("The More Red (Taylor's Version) Chapter", artist)

In [ ]:
create_df_one_album("The More Fearless (Taylor's Version) Chapter", artist)

In [ ]:
create_df_one_album("The More Lover Chapter", artist)

# Output

In [ ]:
# Write recipe outputs
test = dataiku.Dataset("test")
test.write_with_schema(df_complete)